## 환경설정

아래 설치 방법을 참고하여 `uv` 를 설치합니다.

**uv 설치 방법**

```bash
# macOS/Linux
curl -LsSf https://astral.sh/uv/install.sh | sh

# Windows (PowerShell)
irm https://astral.sh/uv/install.ps1 | iex
```

**의존성 설치**

```bash
uv pip install -r requirements.txt
```

환경변수를 가져옵니다.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## MultiServerMCPClient

사전에 `mcp_server_remote.py` 를 실행해둡니다. 터미널을 열고 가상환경이 활성화 되어 있는 상태에서 서버를 실행해 주세요.

> 명령어
```bash
source .venv/bin/activate
python mcp_server_remote.py
```

`async with` 로 일시적인 Session 연결을 생성 후 해제

이제 그럼 Async Session 을 유지하며 도구에 접근하는 방식으로 변경해 보겠습니다.

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from utils import astream_graph
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name="gpt-4o-mini", temperature=0
)

# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        "playwright": {
            # 서버의 포트와 일치해야 합니다.(8005번 포트)
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)


# 2. 명시적으로 연결 초기화 (이 부분이 필요함)
# 초기화
await client.__aenter__()

# 이제 도구가 로드됨
print(client.get_tools())  # 도구가 표시됨

[StructuredTool(name='browser_close', description='Close the page', args_schema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000002031D2F3740>), StructuredTool(name='browser_wait', description='Wait for a specified time in seconds', args_schema={'type': 'object', 'properties': {'time': {'type': 'number', 'description': 'The time to wait in seconds'}}, 'required': ['time'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000002031D2F3920>), StructuredTool(name='browser_resize', description='Resize the browser window', args_schema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'h

langgraph 의 에이전트를 생성합니다.

In [4]:
# 에이전트 생성
agent = create_react_agent(model, client.get_tools())

그래프를 실행하여 결과를 확인합니다.

In [9]:
await astream_graph(agent, {"messages": "1. https://edu.ssafy.com 에 접속해 2. 제대로 메인 홈페이지 화면이 나오는지 확인해 3. (성공/실패, 해당 프로세스 동작 속도, 피드백) 이 항목들에 대해 작성해줘"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
- Ran code:
```js
// Navigate to https://edu.ssafy.com
await page.goto('https://edu.ssafy.com');
```

- Page URL: https://edu.ssafy.com/comm/login/SecurityLoginForm.do
- Page Title: 삼성 청년 SW·AI 아카데미
- Page Snapshot
```yaml
- banner [ref=s1e4]
- heading "SAMSUNG SW·AI ACADEMY FOR YOUTH" [level=1] [ref=s1e12]:
    - img "SAMSUNG SW·AI ACADEMY FOR YOUTH" [ref=s1e13]
- text: 아이디
- textbox "아이디" [ref=s1e18]
- text: 비밀번호
- textbox "비밀번호" [ref=s1e21]
- text: 아이디 저장
- link "로그인" [ref=s1e26]:
    - /url: javascript:void(0);
- list [ref=s1e28]:
    - listitem [ref=s1e29]:
        - link "비밀번호 찾기" [ref=s1e30]:
            - /url: /edu/login/pwdsearch/pwdSearchForm.do
- emphasis [ref=s1e34]: SAMSUNG SW·AI ACADEMY FOR YOUTH
- paragraph [ref=s1e36]: SSAFY에 오신것을 환영합니다.
- contentinfo [ref=s1e38]:
    - text: 개인정보처리방침
    - paragraph [ref=s1e43]: Copyright © SAMSUNG Al

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0'}, id='run-e693d375-7f99-4ad5-be25-f2c819008600'),
 'metadata': {'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent',),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:3fdc5fa1-1a7a-46ed-7136-67458959db9d',
  'checkpoint_ns': 'agent:3fdc5fa1-1a7a-46ed-7136-67458959db9d',
  'ls_provider': 'openai',
  'ls_model_name': 'gpt-4o-mini',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0}}